In [3]:
import pandas as pd

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest,f_classif

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def create_base_train_data(x,y):
    selector=SelectKBest(score_func=f_classif,k=5).fit(x,y)
    x_select=selector.transform(x)
    select_features=selector.get_support(1)
    new_x,new_y=SMOTE(sampling_strategy=0.2,random_state=SEED).fit_resample(x_select,y)
    new_x=PolynomialFeatures(degree=2).fit_transform(new_x)
    return new_x,new_y,select_features

def create_base_test_data(x,feature):
    new_x=x[x.columns[feature]]
    new_x=PolynomialFeatures(degree=2).fit_transform(new_x)
    return new_x

def predict_score(model,x_train,y_train,x_test,y_test,name):
    model.fit(x_train,y_train)
    y_predict=model.predict(x_test)
    f1[name]=f1_score(y_test,y_predict)
    auc_score[name]=roc_auc_score(y_test,y_predict)

SEED=42
df=pd.read_csv("dataset/lm.csv")
x=df.drop(["class","smell"],axis=1)
y=df[["smell"]]
f1={}
auc_score={}

In [4]:
print("x:",x.columns,"\n")
print("y:",y.columns,"\n")
rate=y.value_counts()[1]/y.value_counts()[0]
print(y.value_counts())
print("割合:",rate)

x: Index(['fanin', 'fanout', 'wmc', 'loc', 'returnsQty', 'variablesQty',
       'parametersQty', 'loopQty', 'anonymousClassesQty', 'innerClassesQty'],
      dtype='object') 

y: Index(['smell'], dtype='object') 

smell
0.0      287706
1.0        1342
Name: count, dtype: int64
割合: 0.004664483882852634


In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=SEED)
x_train_base,y_train_base,features=create_base_train_data(x_train,y_train)
x_test_base=create_base_test_data(x_test,features)

print(len(x_train_base))
print(len(y_train_base))


276234
276234


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
name="DT"
DT_prameter={
    "criterion":["gini","entropy"],
    "max_depth":list(range(1,11)),
    "min_samples_split":list(range(2,25)),
    "min_samples_leaf":list(range(1,20)),
    "max_features":["sqrt","log2",None]
    }
DT=DecisionTreeClassifier(random_state=SEED)

DT_model=RandomizedSearchCV(DT, param_distributions=DT_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(DT_model,x_train_base,y_train_base,x_test_base,y_test,name)


In [7]:
name="RF"
RF_prameter={    
    'n_estimators': [100, 200, 300, 400, 500],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
    }
RF=RandomForestClassifier(random_state=SEED)

RF_model=RandomizedSearchCV(RF, param_distributions=RF_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(RF_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\b

In [8]:
name="GBR"
GBR_prameter={
    'n_estimators': [10, 100, 1000],
    'learning_rate': [0.001, 0.01, 0.1],
    'subsample': [0.5, 0.7, 1.0],
    'max_depth': [3, 7, 9]
  }
GBR=GradientBoostingClassifier(random_state=SEED)

GBR_model=RandomizedSearchCV(GBR, param_distributions=GBR_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(GBR_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklear

In [9]:
name="LR"
LR_prameter={
    'solver': ['newton-cg', 'liblinear'],
    'penalty': ['l2'],
    'C': [100, 10, 1.0, 0.1, 0.01],
    }
LR=LogisticRegression(random_state=SEED)

LR_model=RandomizedSearchCV(LR, param_distributions=LR_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(LR_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\optimize.py:319: ConvergenceWarning: newton-cg failed to converge at loss = 0.18554674388425657. Increase the number of iterations.
  warnings.warn(
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\optimize.py:319: ConvergenceWarning: newton-cg failed to conv

In [10]:
name="NB"
NB_model=GaussianNB()

predict_score(NB_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
name="KNN"
KNN_prameter={
    'n_neighbors': range(1, 21, 2),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
    }
KNN=KNeighborsClassifier()

KNN_model=RandomizedSearchCV(KNN, param_distributions=KNN_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(KNN_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neighbors\

In [12]:
name="MLP"
MLP_prameter={
    'hidden_layer_sizes': [(10,),(50,),(100,),(50, 50),(100, 50)],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.5, 1.0],
    'batch_size': [1, 32, 64, 128]
    }
MLP=MLPClassifier(random_state=SEED)

MLP_model=RandomizedSearchCV(MLP, param_distributions=MLP_prameter,cv=StratifiedKFold(n_splits=10),random_state=SEED)
predict_score(MLP_model,x_train_base,y_train_base,x_test_base,y_test,name)


c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sugii syuji\AppData\Lo

In [13]:
print("f1",f1)
print("roc_auc",auc_score)

f1 {'DT': 0.31733333333333336, 'RF': 0.44816053511705684, 'GBR': 0.4583835946924005, 'LR': 0.3696, 'NB': 0.4556354916067146, 'KNN': 0.4131551901336074, 'MLP': 0.0508195131413054}
roc_auc {'DT': np.float64(0.8896209996528801), 'RF': np.float64(0.8326688877143119), 'GBR': np.float64(0.8147697962565138), 'LR': np.float64(0.8800279533553245), 'NB': np.float64(0.8147263263116336), 'KNN': np.float64(0.8320081445521342), 'MLP': np.float64(0.853360697784789)}


In [14]:
for key in f1.keys():
    print(key,":",f1[key]," ",auc_score[key])

DT : 0.31733333333333336   0.8896209996528801
RF : 0.44816053511705684   0.8326688877143119
GBR : 0.4583835946924005   0.8147697962565138
LR : 0.3696   0.8800279533553245
NB : 0.4556354916067146   0.8147263263116336
KNN : 0.4131551901336074   0.8320081445521342
MLP : 0.0508195131413054   0.853360697784789


In [1]:
import csv
data={
"DT" : [0.31733333333333336,0.8896209996528801],
"RF" : [0.44816053511705684,0.8326688877143119],
"GBR" : [0.4583835946924005,0.8147697962565138],
"LR" : [0.3696,0.8800279533553245],
"NB" : [0.4556354916067146,0.8147263263116336],
"KNN" : [0.4131551901336074,0.8320081445521342],
"MLP" : [0.0508195131413054,0.853360697784789],
}

name="lm"
header=["","f1","roc_auc"]
with open("../result/"+name+".csv","w",newline="") as f:
    writer=csv.writer(f)
    writer.writerow(header)
    for key,value in data.items():
        writer.writerow([key]+[value[0]]+[value[1]])